In [6]:
import time
import os
import re
import socket
import datetime
import pyautogui

def return_datetime():
    """
    Returns the date and time in the following format YYYY/MM/DD HH:MM:SS
    """
    date_time = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S')
    return date_time

def packet(obj):
    """
    Converts an obj to bytes in order to send them via TCP/IP, requires obj (any type compatible with str() function)
    """
    obj = str(obj)
    return bytes(obj, 'utf-8')
def change_attenuator(value):
    try:
        #take a screenshot
        im=pyautogui.screenshot()
        #Locate the laser tab
        pyautogui.moveTo(200,200)
        obj = pyautogui.locateOnScreen('laser_tab.bmp') 
        pyautogui.moveTo(obj[0], obj[1], 0.05) #Moves the mouse to the upper left corner of the obj
        pyautogui.moveRel(obj[2]/2, obj[3]/2, 0.05) #Moves the mouse to the middle of the detected obj
        pos = pyautogui.position()
        pyautogui.click(pos[0], pos[1], button = 'left') #Clicks on the tab

        #In laser tab find the att position tab
        pyautogui.moveTo(200,200)
        obj = pyautogui.locateOnScreen('degrees_sp.bmp')
        pyautogui.moveTo(obj[0], obj[1], 0.05) #Moves the mouse to the upper left corner of the obj
        pyautogui.moveRel(10, obj[3] * 3/4, 0.05) #Moves the mouse to the middle in y direction and 10 pixels right
        pos = pyautogui.position()
        pyautogui.click(pos[0], pos[1], button = 'left') #Clicks on the field to modify it
        pyautogui.typewrite(['delete', 'delete','delete', 'delete']) #Deletes current field
        pyautogui.typewrite(value) #Types new attenuator value

        #Finally click the set position
        pyautogui.moveTo(200,200)
        obj = pyautogui.locateOnScreen('set_position.bmp')
        pyautogui.moveTo(obj[0], obj[1], 0.05) #Moves the mouse to the upper left corner of the obj
        pyautogui.moveRel(obj[2]/2, obj[3]/2, 0.05) #Moves the mouse to the middle of the detected obj
        pos = pyautogui.position()
        pyautogui.click(pos[0], pos[1], button = 'left') #Clicks on the field to modify it
        return
    except:
        print("except clause entered in change_attenuator(): returning... ", end = '\r')
        return


def create_server(ip_addr, port, BUFFER_SIZE = 32):
    try:
        TCP_IP = ip_addr
        TCP_PORT = port
        prev_att_val = ''
        #BUFFER_SIZE establishes maximum length per packet
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.bind((TCP_IP, TCP_PORT))
        s.listen(1) #Number of connections that can be queued in current port, max size is 5
        conn, addr = s.accept()
        print("Connection established, address: ", addr)
        while True:
            data = conn.recv(BUFFER_SIZE)
            if not data:
                print("\nConnection ended by client")
                break;
            str_data = str(data)
            string = "{}, data received = {}".format(return_datetime(), str_data)
            print(string, end = '\r')
            if str_data != prev_att_val: #Disable for non live testing
                try:
                    print("changed attenuator to {}".format(str_data))
                    #change_attenuator(str_data) #Moves mouse and keyboard to change the attenuator value
                except:
                    continue
            prev_att_val = str_data
            conn.send(packet(return_datetime()))
        conn.close()
        s.close()
    except:
        print("Except clause entered in create_server(): attempting to close connections...")
        conn.close()
        s.close()
    return


In [11]:
create_server('127.0.0.1', 13536)   


Connection established, address:  ('127.0.0.1', 52076)
changed attenuator to b'0.1'ceived = b'0.1'
changed attenuator to b'6.5'ceived = b'6.5'
changed attenuator to b'1.0'ceived = b'1.0'
changed attenuator to b'6.5'ceived = b'6.5'
changed attenuator to b'1.0'ceived = b'1.0'
2017/10/20 13:26:57, data received = b'1.0'
Connection ended by client
